Perceptron Multi-Capa con PyTorch
=================================

Vamos a ver como implementar un perceptron multicapa (MLP) con una sola capa oculta en torch de varias maneras. Las distintas implementaciones van a ir desde más bajo nivel al más alto, para que se entienda que es lo que está haciendo la librería internamente.

Comencemos por definir el problema y la arquitectura de la red. Vamos a tratar de que la red aprenda la forma general del problema de XOR, es decir, el O-Exclusivo para más de dos variables, o también conocido como el problema de paridad. 
Se lo llama problema de paridad porque es esquivalente a contar la cantidad de 1s en un número binario, y el resultado depende de si esta cantidad fue par o impar. (e.g.  00→0, 01→1, 10→1, 11→0, 1001→0, 111011→1) 
Este problema es especialmente difícil para los modelos de aprendizaje automático porque el cambio en cualquier variable de entrada también cambia la salida.
En este caso vamos a considerar 8 variables de entrada, y en lugar de usar variables binarias (0,1), vamos a usar variables bipolares (-1,1). El valor objetivo lo podemos calcular sencillamente como el producto de las variables de entrada.

Empecemos importando la librería, definiendo algunas variables del problema y la arquitectura, y creando el conjunto de datos.

In [1]:
import torch

P = 100    # Cantidad de instancias de datos.
N = 8      # Cantidad de unidades en la capa de entrada.
H = N+1    # Cantidad de unidades en la capa oculta.
M = 1      # Cantidad de unidades en la capa de salida.

x = torch.randn( P, N).sign()
z = torch.prod( x, dim=1).view(P,1)


Lo siguiente va a ser crear nuestro modelo. Para esto vamos a definir explicitamente las matrices de pesos para las capas oculta y de salida. Va a ser necesario también indicar que queremos que sobre estos tensores se cualculen automáticamente los gradientes.
Adicionalmente vamos a necesitar también un tensor poblado con 1s para concatenar con las otras neuronas para que sirvan de *unidad umbral*.

In [2]:
w1 = torch.randn( N+1, H, requires_grad=True)
w2 = torch.randn( H+1, M, requires_grad=True)

bias = torch.ones( P, 1)


Finalmente el entrenamiento lo vamos a realizar en modo *batch*. En *h* e *y* vamos a tener los resultados de la activación de las unidades de la capa oculta y de salida respectivamente, y en *error* vamos a calcular la suma de las diferencias cuadradas entre la salida y los objetivos.
Este *error* es evidentemente la función de costo que queremos minimizar, y torch utiliza las operaciones necesarias para calcularlo (incluyendo las de *y*) para construir el grafo de computo para poder hacer los gradientes automáticos en *w1* y *w2*.
Los gradientes se calculan al utilizar el método *error.backward()*, y los guarda en *w1.grad* y *w2.grad*. Para poder aplicar estos gradientes a los pesos debemos indicar que esas operaciones no van a formar parte de grafo de computo. Esto es lo que hace con el *no_grad*. Finalmente después de aplicarlos los reseteo con *grad.zero_()*.

In [4]:
lr = 1e-2
t, e = 0, 1.
while e>0.01 and t<9999:
    h = torch.cat( (x,bias), dim=1).mm(w1).tanh()
    y = torch.cat( (h,bias), dim=1).mm(w2).tanh()
    error = (y-z).pow(2).sum()
    error.backward()
    with torch.no_grad():
        w1 -= lr*w1.grad
        w2 -= lr*w2.grad
        w1.grad.zero_()
        w2.grad.zero_()
    e = error.item()/P
    t += 1
    if t%1000==0:
        print(t,e)

1000 0.08005979537963867
2000 0.08005378723144531
3000 0.08004894256591796
4000 0.08004493713378906
5000 0.08004156112670899
6000 0.0800386619567871
7000 0.08003614425659179
8000 0.08003395080566406
9000 0.0800320053100586


Si bien una de las grandes ventajas de **pytorch** es que nos permite tener el control suficiente como para poder hacer todas estas operaciones a mano, también es práctico tener funciones, métodos y clases predefinidas a un nivel más abstracto. Para esto existe el módulo **torch.nn** que implementa varias clases relacionadas directamente con redes neuronales.
En nuestro caso, como lo que nos interesa es implementar un modelo con una arquitectura *feedforward*, podemos utilizar la clase *torch.nn.Sequential*. Esta clase agregar capas cuya activación se propaga solo hacia adelante, secuencialmente.
Para definir los elementos del perceptrón multicapa vamos a usar la clase *torch.nn.Linear* que permite crear las pesos completamente conectados entre dos capas (incluyendo los umbrales), y la clase *torch.nn.Tanh* como función de activación. Notar que por tratarse de una clase, y no una función, *Tanh* está escrito con mayúscula; la función *tanh* existe también dentro de la librería escrita con minúscula.
Finalmente vamos a usar una de las funciones de costo de la librería que es el error cuadrático medio (*MSELoss*), y si queremos mantener el mismo tipo de error que estábamos calculando antes inclusive podemos indicar que el lugar del promedio queremos la suma.

In [5]:
model = torch.nn.Sequential(        # Seq es feedforward.
        torch.nn.Linear( N, H),     # Linear son los pesos.
        torch.nn.Tanh(),            # Tanh es la activacion.
        torch.nn.Linear( H, M),     # Linear incluye los bias.
        torch.nn.Tanh() )

costf = torch.nn.MSELoss( reduction='sum')  # Puedo usar la suma.


Esto me permite simplificar y escribir de forma un poco más clara el ciclo de entrenamiento.  Sin embargo todavía necesito actualizar los pesos con sus correspondiente gradientes a mano. Como estos pesos y gradientes ahora se encuentran en lo profundo del objeto model para accederlos tengo que usar el método *parameters* que me devuelve iterativamente todos los tensores con parámetros entrenables.

In [6]:
lr = 1e-2
for t in range(999):
    y = model( x)                   # Puedo usarlo como funcion.
    model.zero_grad()               # Reseteo los grad antes de back.
    error = costf( y, z)
    error.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad
    if not t%100:
        print( t, error.item()/P)


0 1.1306116485595703
100 0.07468906402587891
200 0.04669327259063721
300 0.04333900928497315
400 0.04209674835205078
500 0.04150144577026367
600 0.04116389274597168
700 0.04094815731048584
800 0.040798578262329105
900 0.040688815116882326


Notar que tanto *model* como *costf* son objetos y sin embargo están siendo utilizados como funciones. Esta es una particularidad de **pytorch** que es práctica pero puede resultar confusa al comienzo.
Otras diferencias con la forma anterior es que ahora podemos resetear los gradientes directamente con *zero_grad* y que, a pesar de que el error sigue siendo un escalar, torch devuelve siempre un tensor, en este caso con un solo elemento, por lo cual para imprimirlo más claramente podemos usar *error.item()*.

Pero para poder aprovechar realmente el potencial de la librería lo mejor es crear una nueva clase con la arquitectura deseada para el modelo. Esto se hace derivando la nueva clase de *torch.nn.Module*. Cuando se crea una clase de este tipo hay que tener en cuenta dos cosas:
1. Cualquier miembro de la clase que también sea derivado de *Module* integrará sus parámetros con los del modelo recursivamente.
2. La respuesta del modelo se determina implementando el método *forward*. (Este es el que es llamado cuando se le pasan parámetros al objeto como si fuera una función)

Veamos como quedaría implementado el perceptron multicapa de esta forma y después aclaramos cada punto.

In [7]:
class mlp( torch.nn.Module):                    # Module para hacer modelos propios.
    def __init__( _, isize, hsize, osize):      # Uso _ en lugar de self.
        super().__init__()
        _.l1 = torch.nn.Linear( isize, hsize)   # Params de modelo.
        _.l2 = torch.nn.Linear( hsize, osize)

    def forward( _, x):
        h = torch.tanh( _.l1( x))               # Grafo de computo.
        y = torch.tanh( _.l2( h))
        return y


Al constructor __init__ le paso como parámetros la dimensión de las capas de entrada, oculta, y salida. Con estos valores creo las conexiones para las capas con *Linear* que quedan como los miembros *l1* y *l2*. Como *Linear* también es derivado de *Module* los parámetro entrenables de estos objetos también van ser parte de este modelo, es decir, son devueltos si se itera usando la función *parameters*.  De la misma forma, si *mlp* fuera utilizado como miembro en otra clase derivada de *Module* sus parámetros entrenables también pasaría a formar parte de esta (i.e. recursivo).

Con el método *forward* se determina la respuesta del modelo al recibir datos de entrada *x*. Esto da bastante libertad para calcular la salida de la red, inclusive hasta utilizando ciclos o condicionales, pero se deben utilizar todas funciones propias de *torch* porque este calculo formará parte de el grafo de cómputo para calcular los gradientes automáticos.

Finalmente, ahora que *torch* va a saber cúales son los parámetros entrenables y cómo actualizarlos, podemos también utilizar uno de los métodos de aprendizaje propios de la librería. En este caso el *Stochastic Gradient Descent*.

In [11]:
model = mlp( N, H, M)
optim = torch.optim.SGD( model.parameters(), lr=0.01)
costf = torch.nn.MSELoss()


Y reescribiendo el ciclo de entrenamiento...

In [12]:
t, E = 0, 1.
while E>=0.01 and t<9999:
    y = model( x)
    optim.zero_grad()                       # Optim sabe que resetear.
    error = costf( y, z)
    error.backward()
    optim.step()                            # step aplica los gradientes.
    E = error.item()
    t += 1
    if t%1000==0:
        print( t, E)


1000 0.8892418146133423
2000 0.8439221978187561
3000 0.620249330997467
4000 0.40158817172050476
5000 0.2635127007961273
6000 0.1668662130832672
7000 0.11692226678133011
8000 0.08941445499658585
9000 0.0742102861404419


En donde puedo seguir siempre los mismos pasos, independientemente del modelo que esté entrenando, la función de costo que esté tratando de minimizar, o la estrategia de minimización que haya elegido. Y los pasos clave son:
1. Obtener una respuesta del modelo: *y = model( x)*
2. Inicializar los gradientes: *optim.zero_grad()*
3. Calcular el costo: *error = costf( y, z)*
4. Calcular los gradientes: *error.backward()*
5. Aplicar los gradientes: *optim.step()*

Esta misma estratégia va a ser central para el entrenamiento de todos los modelos y espero que con el desarrollo paso a paso que fuimos haciendo acá haya quedado claro qué está pasando en cada uno de estos pasos.